# Image Undistortion using Charuco Board 

In [ ]:
# System information:
# - Linux Mint 18.1 Cinnamon 64-bit
# - Python 2.7 with OpenCV 3.2.0

import glob
import json
import os
import pickle
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from cv2 import aruco

In [ ]:
filename_glob_pattern = (
    "/home/fritz/Pictures/olympus_undistortion_test/checkerboard_images/31072024/*.JPG"
)

# ChAruco board variables
CHARUCOBOARD_ROWCOUNT = 8
CHARUCOBOARD_COLCOUNT = 11
SQUARE_LENGTH = 0.015
MARKER_LENGTH = 0.011
aruco_dict = cv2.aruco.DICT_4X4_1000
ARUCO_DICT = cv2.aruco.getPredefinedDictionary(aruco_dict)

parameters = cv2.aruco.DetectorParameters()
parameters.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
detector = cv2.aruco.ArucoDetector(ARUCO_DICT, parameters)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.0001)

In [ ]:
# Create constants to be passed into OpenCV and Aruco methods
CHARUCO_BOARD = cv2.aruco.CharucoBoard(
    (CHARUCOBOARD_COLCOUNT, CHARUCOBOARD_ROWCOUNT),
    SQUARE_LENGTH,
    MARKER_LENGTH,
    ARUCO_DICT,
)

# Create the arrays and variables we'll use to store info like corners and IDs from images processed
corners_all = []  # Corners discovered in all images processed
ids_all = []  # Aruco ids corresponding to corners discovered
image_size = None  # Determined at runtime

# This requires a set of images or a video taken with the camera you want to calibrate
# I'm using a set of images taken with the camera with the naming convention:
# 'camera-pic-of-charucoboard-<NUMBER>.jpg'
# All images used should be the same size, which if taken with the same camera shouldn't be a problem
images = glob.glob(filename_glob_pattern)

# Loop through images glob'ed
for iname in images:
    # Open the image
    img = cv2.imread(iname)
    height, width, _ = img.shape
    # Grayscale the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find aruco markers in the query image
    # corners, ids, _ = aruco.detectMarkers(image=gray, dictionary=ARUCO_DICT)
    corners, ids, _ = detector.detectMarkers(image=gray)

    # Outline the aruco markers found in our query image
    img = aruco.drawDetectedMarkers(image=img, corners=corners)

    # Get charuco corners and ids from detected aruco markers
    response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
        markerCorners=corners, markerIds=ids, image=gray, board=CHARUCO_BOARD
    )

    if charuco_corners is None:
        continue

    # Set the needed parameters to find the refined corners
    winSize = (5, 5)
    zeroZone = (-1, -1)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Calculate the refined corner locations
    charuco_corners = cv2.cornerSubPix(
        gray, charuco_corners, winSize, zeroZone, criteria
    )

    # If a Charuco board was found, let's collect image/corner points
    # Requiring at least 20 squares
    if response > 20:
        # Add these corners and ids to our calibration arrays
        corners_all.append(charuco_corners)
        ids_all.append(charuco_ids)

        # Draw the Charuco board we've detected to show our calibrator the board was properly detected
        img = aruco.drawDetectedCornersCharuco(
            image=img, charucoCorners=charuco_corners, charucoIds=charuco_ids
        )

        # If our image size is unknown, set it now
        if not image_size:
            image_size = gray.shape[::-1]

        # Reproportion the image, maxing width or height at 1000
        proportion = max(img.shape) / 1000.0
        img = cv2.resize(
            img, (int(img.shape[1] / proportion), int(img.shape[0] / proportion))
        )
        # Pause to display each image, waiting for key press
        cv2.imshow("Charuco board", img)
        cv2.waitKey(1)
    else:
        print("Not able to detect a charuco board in image: {}".format(iname))

# Destroy any open CV windows
cv2.destroyAllWindows()

# Make sure at least one image was found
if len(images) < 1:
    # Calibration failed because there were no images, warn the user
    print(
        "Calibration was unsuccessful. No images of charucoboards were found. Add images of charucoboards and use or alter the naming conventions used in this file."
    )
    # Exit for failure
    exit()

# Make sure we were able to calibrate on at least one charucoboard by checking
# if we ever determined the image size
if not image_size:
    # Calibration failed because we didn't see any charucoboards of the PatternSize used
    print(
        "Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards."
    )
    # Exit for failure
    exit()

# Now that we've seen all of our images, perform the camera calibration
# based on the set of points we've discovered
calibration, cameraMatrix, distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
    charucoCorners=corners_all,
    charucoIds=ids_all,
    board=CHARUCO_BOARD,
    imageSize=image_size,
    cameraMatrix=None,
    distCoeffs=None,
)

newcameraMatrix, _ = cv2.getOptimalNewCameraMatrix(
    cameraMatrix, distCoeffs, (width, height), 1, (width, height)
)

# Print matrix and distortion coefficient to the console
print(newcameraMatrix)
print(distCoeffs)

# Save values to be used where matrix+dist is required, for instance for posture estimation
# I save files in a pickle file, but you can use yaml or whatever works for you
f = open("calibration.pckl", "wb")
pickle.dump((newcameraMatrix, cameraMatrix, distCoeffs, rvecs, tvecs), f)
f.close()

# Print to console our success
print("Calibration successful. Calibration file used: {}".format("calibration.pckl"))

## Read out saved values

newcameraMatrix, distCoeffs, rvecs, tvecs

In [ ]:
# Calibration data
newcameraMatrix, cameraMatrix, distCoeffs, rvecs, tvecs = pd.read_pickle(
    "calibration.pckl"
)

# Images
filename_glob_pattern = "/home/fritz/Pictures/olympus_undistortion_test/aruco_measurement_images/Tag_0/*.JPG"

# Output path
output_dir = "/home/fritz/Pictures/olympus_undistortion_test/fish_images/undistorted/"

In [ ]:
# Multical
f = open(
    "/home/fritz/Pictures/olympus_undistortion_test/checkerboard_images/31072024/calibration.json"
)
data = json.load(f)
cameraMatrix = np.array(data["cameras"]["cam1"]["K"])
distCoeffs = np.array(data["cameras"]["cam1"]["dist"]).reshape(1, 5)

In [ ]:
images = glob.glob(filename_glob_pattern)

cv2.namedWindow("Undistorted Image", cv2.WINDOW_NORMAL)

if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)

for img in images:
    filename = output_dir + Path(img).stem + "_undistorted" + Path(img).suffix
    print(filename)
    img = cv2.imread(img)

    # Undistort the image
    h, w = img.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
        cameraMatrix, distCoeffs, (w, h), 1, (w, h)
    )
    undistorted_image = cv2.undistort(
        img, cameraMatrix, distCoeffs, None, new_camera_matrix
    )

    # Crop the image (if desired, based on ROI)
    x, y, w, h = roi
    undistorted_image = undistorted_image[y : y + h, x : x + w]

    cv2.imwrite(filename, undistorted_image)
    cv2.imshow("Undistorted Image", undistorted_image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

## Warp Transform using Aruco Tag

In [ ]:
import glob

import cv2
import numpy as np


def rectify_image_using_aruco(
    image_path, cameraMatrix, distCoeffs, aruco_dict_type=cv2.aruco.DICT_4X4_100
):
    # Load the image
    img = cv2.imread(image_path)

    # Undistort the image
    h, w = img.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
        cameraMatrix, distCoeffs, (w, h), 1, (w, h)
    )
    undistorted_image = cv2.undistort(
        img, cameraMatrix, distCoeffs, None, new_camera_matrix
    )

    # Crop the image (if desired, based on ROI)
    x, y, w, h = roi
    img = undistorted_image[y : y + h, x : x + w]

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load the dictionary that was used to generate the markers
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
    # Initialize the detector parameters using default values
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

    # Detect the markers in the image
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(
        gray, aruco_dict, parameters=parameters
    )

    if ids is not None:
        # Assuming we have at least one marker detected, we will use the first detected marker
        corners = corners[0].reshape((4, 2))

        # Define the destination points for the perspective transform
        (top_left, top_right, bottom_right, bottom_left) = corners

        width = max(
            int(np.linalg.norm(bottom_right - bottom_left)),
            int(np.linalg.norm(top_right - top_left)),
        )
        height = max(
            int(np.linalg.norm(top_right - bottom_right)),
            int(np.linalg.norm(top_left - bottom_left)),
        )

        dst_pts = np.array(
            [
                [img.shape[1] - (width - 1), img.shape[0] / 4],
                [img.shape[1], img.shape[0] / 4],
                [img.shape[1], (img.shape[0] / 4) + (height - 1)],
                [img.shape[1] - (width - 1), (img.shape[0] / 4) + (height - 1)],
            ],
            dtype="float32",
        )

        # Get the perspective transform matrix
        M = cv2.getPerspectiveTransform(corners, dst_pts, cv2.WARP_INVERSE_MAP)

        # Apply the perspective transformation to get the rectified image
        rectified_img = cv2.warpPerspective(
            img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR
        )

        return rectified_img
    else:
        print("No ArUco markers detected.")
        return img

In [ ]:
# Load the predefined dictionary
aruco_dict = cv2.aruco.DICT_4X4_1000
dictionary = cv2.aruco.getPredefinedDictionary(aruco_dict)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Measurement images containing charuco tag
charuco_measurement_images = sorted(
    glob.glob(
        "/home/fritz/Pictures/olympus_undistortion_test/aruco_measurement_images/Tag_0/labeled/*.JPG"
    )
)

# Output dir
output_dir = "/home/fritz/Pictures/olympus_undistortion_test/aruco_measurement_images/Tag_0/undistored_rectified/"
if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)

# Multical Calibration Parameters
f = open(
    "/home/fritz/Pictures/olympus_undistortion_test/checkerboard_images/31072024/calibration.json"
)
data = json.load(f)
cameraMatrix = np.array(data["cameras"]["cam1"]["K"])
distCoeffs = np.array(data["cameras"]["cam1"]["dist"]).reshape(1, 5)

In [ ]:
for img in charuco_measurement_images:
    frame = cv2.imread(img)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect markers in the image
    # corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray)

    # If markers are detected
    if ids is not None:
        # Draw the markers on the frame
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        rectified_image = rectify_image_using_aruco(img, cameraMatrix, distCoeffs)

        ## Save to file
        filename = output_dir + Path(img).stem + "_rectified" + Path(img).suffix
        cv2.imwrite(filename, rectified_image)

        cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
        cv2.imshow("Frame", rectified_image)
        cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

# Segment Anything 2

In [2]:
import torch, grp, pwd, os, subprocess, gc
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
1
AMD Radeon Graphics


In [4]:
from os import putenv
from ultralytics import ASSETS, SAM

putenv("HSA_OVERRIDE_GFX_VERSION", "11.0.0") # override and trick to seem like using 10.3.0
putenv("PYTORCH_ROCM_ARCH", "gfx1100") # override and trick to seem like using 10.3.0

# putenv("HSA_OVERRIDE_GFX_VERSION", "10.3.0") # override and trick to seem like using 10.3.0
# putenv("PYTORCH_ROCM_ARCH", "gfx1030") # override and trick to seem like using 10.3.0
# putenv("HIP_VISIBLE_DEVICES","0")
putenv("PYTORCH_HIP_ALLOC_CONF","garbage_collection_threshold:0.6,max_split_size_mb:128")
# putenv("COMMANDLINE_ARGS", "--opt-split-attention --no-half-vae --disable-nan-check --autolaunch --listen")
putenv("COMMANDLINE_ARGS", "--upcast-sampling --opt-sub-quad-attention")

# Load a model
model = SAM("/home/fritz/Downloads/sam2_t.pt")

# Display model information (optional)
model.info()

# Run inference
# model("path/to/video.mp4")
results = model(ASSETS / "/home/fritz/Pictures/olympus_undistortion_test/aruco_measurement_images/Tag_0/undistored_rectified/RB251_Rank1_00009_P7290010_rectified.JPG", device="cuda")
# model(ASSETS / , device="cpu)

# Clean up
del model
gc.collect()
torch.cuda.empty_cache()

Model summary: 422 layers, 38,945,986 parameters, 38,945,986 gradients



RuntimeError: CUDA error: HIPBLAS_STATUS_ALLOC_FAILED when calling `hipblasCreate(handle)`

In [ ]:
import cv2

# Visualize the detection
img = results[0].plot()  # This plots the detections on the image

# Convert BGR to RGB (OpenCV uses BGR by default)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.namedWindow('Detection Results', cv2.WINDOW_NORMAL)
cv2.imshow('Detection Results', img_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import torch, grp, pwd, os, subprocess
devices = []
try:
	print("\n\nChecking ROCM support...")
	result = subprocess.run(['rocminfo'], stdout=subprocess.PIPE)
	cmd_str = result.stdout.decode('utf-8')
	cmd_split = cmd_str.split('Agent ')
	for part in cmd_split:
		item_single = part[0:1]
		item_double = part[0:2]
		if item_single.isnumeric() or item_double.isnumeric():
			new_split = cmd_str.split('Agent '+item_double)
			device = new_split[1].split('Marketing Name:')[0].replace('  Name:                    ', '').replace('\n','').replace('                  ','').split('Uuid:')[0].split('*******')[1]
			devices.append(device)
	if len(devices) > 0:
		print('GOOD: ROCM devices found: ', len(devices))
	else:
		print('BAD: No ROCM devices found.')

	print("Checking PyTorch...")
	x = torch.rand(5, 3)
	has_torch = False
	len_x = len(x)
	if len_x == 5:
		has_torch = True
		for i in x:
			if len(i) == 3:
				has_torch = True
			else:
				has_torch = False
	if has_torch:
		print('GOOD: PyTorch is working fine.')
	else:
		print('BAD: PyTorch is NOT working.')


	print("Checking user groups...")
	user = os.getlogin()
	groups = [g.gr_name for g in grp.getgrall() if user in g.gr_mem]
	gid = pwd.getpwnam(user).pw_gid
	groups.append(grp.getgrgid(gid).gr_name)
	if 'render' in groups and 'video' in groups:
		print('GOOD: The user', user, 'is in RENDER and VIDEO groups.')
	else:
		print('BAD: The user', user, 'is NOT in RENDER and VIDEO groups. This is necessary in order to PyTorch use HIP resources')

	if torch.cuda.is_available():
		print("GOOD: PyTorch ROCM support found.")
		t = torch.tensor([5, 5, 5], dtype=torch.int64, device='cuda')
		print('Testing PyTorch ROCM support...')
		if str(t) == "tensor([5, 5, 5], device='cuda:0')":
			print('Everything fine! You can run PyTorch code inside of: ')
			for device in devices:
				print('---> ', device)
	else:
		print("BAD: PyTorch ROCM support NOT found.")
except:
	print('Cannot find rocminfo command information. Unable to determine if AMDGPU drivers with ROCM support were installed.')
